In [50]:
# read elpis output file to df
from io import StringIO
import pandas as pd


In [65]:
import xml.etree.ElementTree as ET 
#tree = ET.parse("/users/irisz/downloads/test_transcriptions/ckm009-2023-05-26-Žminj_01.eaf")
tree = ET.parse("/users/irisz/downloads/ckm009-2023-05-26-Žminj_02.eaf")
tree

In [66]:
root = tree.getroot() 
root

<Element 'ANNOTATION_DOCUMENT' at 0x7f9980c36860>

In [67]:
df_1 = pd.DataFrame()
TIME_SLOT_ID = []
TIME_VALUE = []
for child in root[1]:
    #print(child.tag, child.attrib)
    TIME_SLOT_ID.append(child.attrib['TIME_SLOT_ID'])
    TIME_VALUE.append(child.attrib['TIME_VALUE'])
df_1['TIME_SLOT_ID'] = TIME_SLOT_ID
df_1['TIME_VALUE'] = TIME_VALUE
df_1

,TIME_SLOT_ID,TIME_VALUE
0,ts2,70
1,ts3,669
2,ts4,2400
3,ts5,2690
4,ts6,2690
...,...,...
2707,ts2709,580860
2708,ts2710,580880
2709,ts2711,581230
2710,ts2712,581260


In [68]:
df_2 = pd.DataFrame()
ANNOTATION_ID = []
TIME_SLOT_REF1 = []
TIME_SLOT_REF2 = []
ANNOTATION_VALUE = []

for child in root[2]:
    #print(child[0].tag, child[0].attrib)
    #print(child[0][0].text)
    ANNOTATION_ID.append(child[0].attrib['ANNOTATION_ID'])
    TIME_SLOT_REF1.append(child[0].attrib['TIME_SLOT_REF1'])
    TIME_SLOT_REF2.append(child[0].attrib['TIME_SLOT_REF2'])
    ANNOTATION_VALUE.append(child[0][0].text)
    
df_2['ANNOTATION_ID'] = ANNOTATION_ID
df_2['TIME_SLOT_REF1'] = TIME_SLOT_REF1
df_2['TIME_SLOT_REF2'] = TIME_SLOT_REF2
df_2['ANNOTATION_VALUE'] = ANNOTATION_VALUE
df_2

,ANNOTATION_ID,TIME_SLOT_REF1,TIME_SLOT_REF2,ANNOTATION_VALUE
0,a2,ts2,ts3,dobro
1,a4,ts4,ts5,san
2,a6,ts6,ts7,van
3,a8,ts8,ts9,zi
4,a10,ts10,ts11,rike
...,...,...,...,...
1351,a2704,ts2704,ts2705,i
1352,a2706,ts2706,ts2707,vaš
1353,a2708,ts2708,ts2709,jako
1354,a2710,ts2710,ts2711,to


In [69]:
df_3 = pd.DataFrame()
ANNOTATION_ID = []
ANNOTATION_REF = []
ANNOTATION_VALUE = []

for child in root[3]:
    #print(child[0].tag, child[0].attrib)
    #print(child[0][0].text)
    ANNOTATION_ID.append(child[0].attrib['ANNOTATION_ID'])
    ANNOTATION_REF.append(child[0].attrib['ANNOTATION_REF'])
    ANNOTATION_VALUE.append(child[0][0].text)
    
df_3['ANNOTATION_ID'] = ANNOTATION_ID
df_3['ANNOTATION_REF'] = ANNOTATION_REF
df_3['ANNOTATION_VALUE'] = ANNOTATION_VALUE
df_3

,ANNOTATION_ID,ANNOTATION_REF,ANNOTATION_VALUE
0,a3,a2,1.00
1,a5,a4,1.00
2,a7,a6,0.44
3,a9,a8,0.99
4,a11,a10,0.66
...,...,...,...
1351,a2705,a2704,0.77
1352,a2707,a2706,0.58
1353,a2709,a2708,0.93
1354,a2711,a2710,1.00


In [79]:
# merge all dfs

df_all = pd.DataFrame()
start = []
end = []
transcription = []
confidence = []

for i in range(len(df_2)):
    trans_text = df_2['ANNOTATION_VALUE'][i]
    
    start_id = df_2['TIME_SLOT_REF1'][i]
    end_id = df_2['TIME_SLOT_REF2'][i]
    anno_id = df_2['ANNOTATION_ID'][i]
    
    
    start_time = df_1.loc[df_1['TIME_SLOT_ID'] == start_id]['TIME_VALUE'].tolist()[0]
    end_time = df_1.loc[df_1['TIME_SLOT_ID'] == end_id]['TIME_VALUE'].tolist()[0]
    confidence_value = df_3.loc[df_3['ANNOTATION_REF'] == anno_id]['ANNOTATION_VALUE'].tolist()[0]
    
    transcription.append(trans_text)
    start.append(float(start_time)/1000.0)
    end.append(float(end_time)/1000.0)
    confidence.append(confidence_value)
    
df_all['start'] = start
df_all['end'] = end
df_all['transcription'] = transcription
df_all['confidence'] = confidence
df_all

,start,end,transcription,confidence
0,0.07,0.669,dobro,1.00
1,2.40,2.690,san,1.00
2,2.69,2.810,van,0.44
3,2.81,3.190,zi,0.99
4,3.23,3.650,rike,0.66
...,...,...,...,...
1351,579.81,579.920,i,0.77
1352,579.98,580.400,vaš,0.58
1353,580.40,580.860,jako,0.93
1354,580.88,581.230,to,1.00


In [73]:
df_all.to_csv("/users/irisz/downloads/ckm009-2023-05-26-Žminj_02.csv",index=False)

In [82]:
df_all

,start,end,transcription,confidence
0,0.07,0.669,dobro,1.00
1,2.40,2.690,san,1.00
2,2.69,2.810,van,0.44
3,2.81,3.190,zi,0.99
4,3.23,3.650,rike,0.66
...,...,...,...,...
1351,579.81,579.920,i,0.77
1352,579.98,580.400,vaš,0.58
1353,580.40,580.860,jako,0.93
1354,580.88,581.230,to,1.00


In [75]:
df_all['transcription'].tolist()

['dobro',
 'san',
 'van',
 'zi',
 'rike',
 'nekako',
 'kat',
 'bi',
 'bilo',
 'va',
 'vrtu',
 'čeka',
 'mesto',
 'već',
 'igrat',
 'rekla',
 'da',
 'su',
 'jako',
 'farbi',
 'preveć',
 'su',
 'zijala',
 'kako',
 'nu',
 'to',
 'bilo',
 'eh',
 'da',
 'neki',
 'ustali',
 'kakovu',
 'i',
 'zvizde',
 'i',
 'ja',
 'mislin',
 'sve',
 'i',
 'spal',
 'i',
 'ni',
 'je',
 'i',
 'i',
 'je',
 'smo',
 'hodili',
 'nanke',
 'i',
 'oni',
 'ki',
 'put',
 'su',
 'neš',
 'ideja',
 'to',
 'poć',
 'i',
 'puti',
 'kakovo',
 'pijat',
 'čo',
 'ili',
 'zad',
 'kuće',
 'mali',
 'teplije',
 'sat',
 'tako',
 'pogotovo',
 'tamo',
 'i',
 'je',
 'isto',
 'sir',
 'i',
 'ku',
 'bi',
 'onda',
 'to',
 'smo',
 'pojeli',
 'poć',
 'tu',
 'bi',
 'reć',
 'sarme',
 'nisan',
 'ja',
 'baš',
 'imela',
 'lačna',
 'ali',
 'kad',
 'god',
 'bi',
 'i',
 'i',
 'bila',
 'imela',
 'bili',
 'bilo',
 'ja',
 'nauku',
 'o',
 'ća',
 'če',
 'pa',
 'tamo',
 'seli',
 'isto',
 'pune',
 'ni',
 'niku',
 'poznala',
 'se',
 'najveće',
 'te',
 'hrvack

In [87]:
# get force-aligned wer

# correct df
import numpy as np
def get_audio_df(textgrid_dir):
    try:
        with open(textgrid_dir, 'r', encoding='utf-16') as file:
            lines = [line.rstrip() for line in file]
    except:
        with open(textgrid_dir, 'r') as file:
            lines = [line.rstrip() for line in file]
       
    text_list = []
    xmin_list = []
    xmax_list = []
    interval_list = []
    for line in lines[14:]:  #informations needed begin on the 9th lines
        if 'text =' in line:
            line = line.split('"')[1]
            if line != "":
                text_list.append(line)
                #print (len(text_list), len(xmin_list), len(xmax_list), len(interval_list), line, interval_list[-2:])
            else:
                xmin_list = xmin_list[:-1]
                xmax_list = xmax_list[:-1]
                interval_list = interval_list[:-1]
        if 'xmin' in line:
            time_min = line.split('=')[1].strip(' ')
            xmin_list.append(time_min)
        if 'xmax' in line:
            time_max = line.split('=')[1].strip(' ')
            xmax_list.append(time_max)
        if 'intervals [' in line:
            interval = line.split('[')[1].strip(']:')
            interval_list.append(interval)
        if 'intervals: size' in line:
            xmin_list = xmin_list[:-1]
            xmax_list = xmax_list[:-1]
    xmin_list = [float(item) for item in xmin_list]
    xmax_list = [float(item) for item in xmax_list]
    interval_list = [int(item) for item in interval_list]

    df = pd.DataFrame()
    df['text'] = text_list
    df['xmin'] = xmin_list
    df['xmax'] = xmax_list
    df['interval'] = interval_list
    df['tier'] = get_interval(df['interval'].tolist())
    df['start'] = df['xmin'].apply(lambda x: int(np.floor(x*16000)))
    df['end'] = df['xmax'].apply(lambda x: int(np.ceil(x*16000)))
    return df
def get_interval(time_list):
    interval_all = []
    interval_curr = 1
    for i in range(len(time_list)):
        if i == 0:
            interval_all.append(1)
        else:
            if time_list[i] > time_list[i - 1]:
                interval_all.append(interval_curr)
                continue
            else:
                interval_curr = interval_curr + 1
                interval_all.append(interval_curr)
    return interval_all

In [89]:
import re

def clean_text(text):
    if text == 'x':
        return ''
    # remove () pattern, will leave a ) to be removed
    text = re.sub(r'\(([^)]+)\)', '', text)
    special_char_list = [',', '.', '?', '#', '@', '-', ')']
    for char in special_char_list:
        text = text.replace(char, '')
    # change all text to lower cases
    text = text.lower()
    return text

def get_clean_word_list(text):
    new_text = clean_text(text)
    word_list = new_text.split(' ')
    word_list_clean = list(filter(None, word_list))
    return word_list_clean
    
clean_text('((another speaker)) hhuhhh (dddfd) Man ja još par pitanji, ev...')

' hhuhhh  man ja još par pitanji ev'

In [101]:
df_bench_1['text_clean'][1] == []

True

In [136]:

textgrid_dir = "/users/irisz/downloads/elic_test/ckm009-2023-05-26-Žminj_02.TextGrid"
df_bench = get_audio_df(textgrid_dir)
df_bench['text_clean'] = df_bench['text'].apply(lambda x: get_clean_word_list(x))
df_bench_1 = df_bench[df_bench['tier'].isin([1,2])]
df_bench_1

,text,xmin,xmax,interval,tier,start,end,text_clean
0,Dobro.,0.061304,0.586118,2,1,980,9378,[dobro]
1,((breath)),0.586118,0.923131,3,1,9377,14771,[]
2,"@ kad ti @- spomenula si i prietele, tako",0.923131,4.116103,4,1,14770,65858,"[kad, ti, spomenula, si, i, prietele, tako]"
3,((breath)),4.116103,4.352784,5,1,65857,69645,[]
4,kako je bilo ovaj duoći tako iz jenega manjega...,4.352784,7.829583,6,1,69644,125274,"[kako, je, bilo, ovaj, duoći, tako, iz, jenega..."
...,...,...,...,...,...,...,...,...
173,Te druge fešti baren imajo nešto ((breath)) tr...,561.379230,575.416580,141,2,8982067,9206666,"[te, druge, fešti, baren, imajo, nešto, tradic..."
174,((breath)),575.416580,575.797327,142,2,9206665,9212758,[]
175,"a tamo samo te ((breath)) štraci, ti štrafanići",575.797327,579.085003,143,2,9212757,9265361,"[a, tamo, samo, te, štraci, ti, štrafanići]"
176,ni mi baš jako to više,579.401435,581.825541,145,2,9270422,9309209,"[ni, mi, baš, jako, to, više]"


In [142]:
df_bench_1 = df_bench_1[df_bench_1.text_clean.astype(bool)]
df_bench_1 = df_bench_1[['text_clean', 'xmin', 'xmax']]
df_bench_1 = df_bench_1.reset_index()
df_bench_1

,index,text_clean,xmin,xmax
0,0,[dobro],0.061304,0.586118
1,2,"[kad, ti, spomenula, si, i, prietele, tako]",0.923131,4.116103
2,4,"[kako, je, bilo, ovaj, duoći, tako, iz, jenega...",4.352784,7.829583
3,5,"[si, rekla, da, su, ti, prieteli, jako, falili...",7.829583,15.966789
4,6,"[š, njima]",16.276344,17.197341
...,...,...,...,...
114,170,"[i, na, kraje, se, to, pretvorilo, va, samo, g...",554.664054,558.723308
115,171,"[ali, va, štracarije]",558.723308,560.535199
116,173,"[te, druge, fešti, baren, imajo, nešto, tradic...",561.379230,575.416580
117,175,"[a, tamo, samo, te, štraci, ti, štrafanići]",575.797327,579.085003


In [143]:
# candidate row range to acess df_all
cand_row_min = 0
cand_max_max = len(df_bench_1)

merge_loc = {}

for i in range(len(df_all)):
    # get row time range
    start_curr = df_all['start'][i]
    end_curr = df_all['end'][i]
    for j in range(cand_row_min, cand_max_max):
        # if word window land in the sentence one, put in the merge info
        end_merge_curr = df_bench_1['xmax'][j]
        if start_curr < end_merge_curr:
            merge_loc[i] = j
            cand_row_min = j
            break
merge_loc    

{0: 0,
 1: 1,
 2: 1,
 3: 1,
 4: 1,
 5: 1,
 6: 2,
 7: 2,
 8: 2,
 9: 2,
 10: 2,
 11: 2,
 12: 2,
 13: 2,
 14: 2,
 15: 3,
 16: 3,
 17: 3,
 18: 3,
 19: 3,
 20: 3,
 21: 3,
 22: 3,
 23: 3,
 24: 3,
 25: 3,
 26: 3,
 27: 4,
 28: 5,
 29: 5,
 30: 5,
 31: 5,
 32: 5,
 33: 5,
 34: 5,
 35: 6,
 36: 6,
 37: 6,
 38: 6,
 39: 6,
 40: 6,
 41: 6,
 42: 6,
 43: 6,
 44: 6,
 45: 6,
 46: 6,
 47: 6,
 48: 6,
 49: 6,
 50: 6,
 51: 6,
 52: 6,
 53: 6,
 54: 6,
 55: 6,
 56: 6,
 57: 6,
 58: 6,
 59: 6,
 60: 6,
 61: 6,
 62: 6,
 63: 6,
 64: 6,
 65: 6,
 66: 6,
 67: 6,
 68: 6,
 69: 6,
 70: 6,
 71: 6,
 72: 6,
 73: 6,
 74: 6,
 75: 6,
 76: 6,
 77: 6,
 78: 6,
 79: 6,
 80: 6,
 81: 6,
 82: 6,
 83: 6,
 84: 6,
 85: 6,
 86: 6,
 87: 6,
 88: 6,
 89: 6,
 90: 6,
 91: 6,
 92: 6,
 93: 6,
 94: 6,
 95: 6,
 96: 6,
 97: 6,
 98: 6,
 99: 6,
 100: 6,
 101: 6,
 102: 6,
 103: 6,
 104: 6,
 105: 6,
 106: 6,
 107: 6,
 108: 6,
 109: 6,
 110: 6,
 111: 6,
 112: 6,
 113: 6,
 114: 6,
 115: 6,
 116: 6,
 117: 6,
 118: 6,
 119: 6,
 120: 6,
 121: 6,
 122: 6,
 12

In [144]:
merge_loc[1355]

118

In [145]:
df_all['index'] = df_all.index
df_all['merge_loc'] = df_all['index'].apply(lambda x: merge_loc[x])
df_all

,start,end,transcription,confidence,index,merge_loc
0,0.07,0.669,dobro,1.00,0,0
1,2.40,2.690,san,1.00,1,1
2,2.69,2.810,van,0.44,2,1
3,2.81,3.190,zi,0.99,3,1
4,3.23,3.650,rike,0.66,4,1
...,...,...,...,...,...,...
1351,579.81,579.920,i,0.77,1351,118
1352,579.98,580.400,vaš,0.58,1352,118
1353,580.40,580.860,jako,0.93,1353,118
1354,580.88,581.230,to,1.00,1354,118


In [ ]:
# merge df_bench_1 and df_all


In [146]:
df_bench_1['index'] = df_bench_1.index
df_bench_1['transcription'] = df_bench_1['index'].apply(lambda x: df_all[df_all['merge_loc']==x]['transcription'].tolist())
df_bench_1

,index,text_clean,xmin,xmax,transcription
0,0,[dobro],0.061304,0.586118,[dobro]
1,1,"[kad, ti, spomenula, si, i, prietele, tako]",0.923131,4.116103,"[san, van, zi, rike, nekako]"
2,2,"[kako, je, bilo, ovaj, duoći, tako, iz, jenega...",4.352784,7.829583,"[kat, bi, bilo, va, vrtu, čeka, mesto, već, ig..."
3,3,"[si, rekla, da, su, ti, prieteli, jako, falili...",7.829583,15.966789,"[rekla, da, su, jako, farbi, preveć, su, zijal..."
4,4,"[š, njima]",16.276344,17.197341,[eh]
...,...,...,...,...,...
114,114,"[i, na, kraje, se, to, pretvorilo, va, samo, g...",554.664054,558.723308,"[ima, trajset, to, pretvori, ala, va, samo, vi..."
115,115,"[ali, va, štracarije]",558.723308,560.535199,"[ali, dva, prasca, je]"
116,116,"[te, druge, fešti, baren, imajo, nešto, tradic...",561.379230,575.416580,"[mat, je, druge, friški, baren, ima, ja, nešto..."
117,117,"[a, tamo, samo, te, štraci, ti, štrafanići]",575.797327,579.085003,"[a, tamo, samo, te, špijat, sjetiš, ten, san, ..."


In [147]:
from Bio import pairwise2
from thefuzz import fuzz
import numpy as np

def get_aligned_list(l1, l2):
    # get aligned list, along with wer type and fuzz match score
    alignments = pairwise2.align.globalxx(l1, 
                                      l2,
                                      gap_char=['-']
                                     )
    l1_align = alignments[0].seqA
    l2_align = alignments[0].seqB
    score = []
    for i in range(len(l1_align)):
        score.append(word_match_score(l1_align[i], l2_align[i]))
    return l1_align, l2_align, score


def word_match_score(t1, t2):
    if t1 == '-' or t2 == '-':
        return 0
    else:
        score = fuzz.ratio(t1, t2)
        return score

def get_matched(t1, t2):
    #print (t1, t2)
    if t1 == t2:
            return 1
    else:
        return 0
    
def get_unmatched_cand(match_list, idx):
    cand = []
    idx_org = idx
    if match_list[idx] == 1:
        return [idx]
    else:
        while match_list[idx] != 1 and idx > 0:
            cand.append(idx)
            idx = idx - 1 # before
        idx = idx_org
        while match_list[idx] != 1 and idx < len(match_list) -1:
            cand.append(idx)
            idx = idx + 1 # after
        return list(set(cand))
    
def get_can_score(idx,match_cand, df_align):
    if len(match_cand) == 1:
        return [100]
    score_all = []
    for cand in match_cand:
        t1 = df_align['manual'][idx]
        t2 = df_align['model'][cand]
        score = word_match_score(t1, t2)
        score_all.append(score)
    return score_all

def get_fuzzy_cand(match_cand, match_cand_score, df_align):
    max_cand = np.max(match_cand_score)
    if max_cand > 60:
        # fuzzy matched
        max_cand_id = match_cand[match_cand_score.index(max_cand)]
        #print (max_cand_id)
        word = df_align['model'][max_cand_id]
        return word, max_cand, max_cand_id
    else:
        return '-', 0, '-'
    
def get_error_type(t1, t2):
    if (t1 != '-') and (t2 != '-') and (t1 != t2):
        return 's'
    if (t1 != '-') and (t2 == '-'):
        return 'd'
    if (t1 == '-') and (t2 != '-'):
        return 'i'
    if (t1 != '-') and (t2 != '-') and (t1 == t2):
        return 'c'
    if t1 == '':
        return ""

/Users/irisz/opt/miniconda3/lib/python3.9/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [161]:
def get_df_align_and_wer_info(df, file_name):
    save_dir = '/users/irisz/downloads/elpis_result/' + file_name + '_wer.csv'
    manual_all = []
    model_all = []
    model_fuzzy_all = []
    error_type_all = []
    wer_all = []
    for id in range(len(df)):
        #text1 = df['text'][id]
        #text2 = df[model_name][id] # already have list
        #l1 = get_clean_word_list(text1)
        #l2 = get_clean_word_list(text2)
        l1 = df['text_clean'][id]
        l2 = df['transcription'][id]
        #if l1 == []:
            #l1 = [""]
        if l2 == []:
            l2 = [""]
        #print (l1, l2)
        df_align = pd.DataFrame(columns = ['manual', 'model'])
        #print (l1, l2)
        df_align['manual'] = get_aligned_list(l1, l2)[0]
        df_align['model'] = get_aligned_list(l1, l2)[1]
        df_align['score'] = get_aligned_list(l1, l2)[2]
        df_align['match'] = df_align.apply(lambda x:get_matched(x.manual, x.model), axis = 1)
        df_align['idx'] = df_align.index
        df_align['match_cand'] = df_align.apply(lambda x: get_unmatched_cand(df_align['match'].tolist(), x.idx), axis = 1)
        df_align['match_cand_score'] = df_align.apply(lambda x: get_can_score(x.idx, x.match_cand, df_align), axis = 1)
        df_align['model_fuzzy'] = df_align.apply(lambda x: get_fuzzy_cand(x.match_cand, x.match_cand_score, df_align)[0], axis = 1)
        df_align['model_fuzzy_id'] = df_align.apply(lambda x: get_fuzzy_cand(x.match_cand, x.match_cand_score, df_align)[2], axis = 1)
        df_align['model_fuzzy_score'] = df_align.apply(lambda x: get_fuzzy_cand(x.match_cand, x.match_cand_score, df_align)[1], axis = 1)
        df_align['error_type'] = df_align.apply(lambda x: get_error_type(x.manual, x.model), axis = 1)
        sub_id = df_align[(df_align['error_type']=='d') & (df_align['model_fuzzy_id'] != '-')]['model_fuzzy_id'].tolist()
        del_sub_id = df_align[(df_align['error_type']=='d') & (df_align['model_fuzzy_id'] != '-')].index.tolist()
        error_type = df_align['error_type'].tolist()
        for id in del_sub_id:
            error_type[id] = 's'
        df_align['error_type_correct'] = error_type
        df_align_clean = df_align[~df_align.index.isin(sub_id)]

        manual_curr = df_align_clean['manual'].tolist()
        model_fuzzy_curr  = df_align_clean['model_fuzzy'].tolist()
        error_type  = df_align_clean['error_type_correct'].tolist()
        model_curr = df_align_clean['model'].tolist()
        wer_curr = get_wer_from_df(df_align_clean)
        
        manual_all = manual_all + manual_curr
        model_fuzzy_all = model_fuzzy_all + model_fuzzy_curr
        error_type_all = error_type_all + error_type
        model_all = model_all + model_curr
        wer_all.append(wer_curr)
    df_all = pd.DataFrame(columns = ['manual', 'model', 'model_fuzzy', 'error_type'])
    df_all['manual'] = manual_all
    df_all['model'] = model_all
    df_all['model_fuzzy'] = model_fuzzy_all
    df_all['error_type'] = error_type_all
    
    df_all.to_csv(save_dir)
    return wer_all

def get_wer_from_df(df):
    s = len(df[df['error_type_correct'] == 's'])
    d = len(df[df['error_type_correct'] == 'd'])
    i = len(df[df['error_type_correct'] == 'i'])
    c = len(df[df['error_type_correct'] == 'c'])
    n = s + d + c
    if n == 0:
        wer = np.inf
    else:
        wer = (s + d + i)/n * 100
    return wer,s,d,i,c,n

In [157]:
get_df_align_and_wer_info(df_bench_1, 'ckm009-2023-05-26-Žminj_02')

['dobro'] ['dobro']
['kad', 'ti', 'spomenula', 'si', 'i', 'prietele', 'tako'] ['san', 'van', 'zi', 'rike', 'nekako']
['kako', 'je', 'bilo', 'ovaj', 'duoći', 'tako', 'iz', 'jenega', 'manjega', 'mesta', 'u', 'veći', 'grad'] ['kat', 'bi', 'bilo', 'va', 'vrtu', 'čeka', 'mesto', 'već', 'igrat']
['si', 'rekla', 'da', 'su', 'ti', 'prieteli', 'jako', 'falili', 'pole', 'jeno', 'vreme', 'si', 'rekla', 'da', 'kad', 'si', 'preveć', 'študijala', 'kako', 'je', 'onda', 'to', 'bilo', 'pole', 'se', 'jopet', 'prilagodit'] ['rekla', 'da', 'su', 'jako', 'farbi', 'preveć', 'su', 'zijala', 'kako', 'nu', 'to', 'bilo']
['š', 'njima'] ['eh']
['da', 'li', 'ti', 'je', 'ustalo', 'kakovo', 'sjećanje', 'iz', 'izlaska', 'za', 'kad', 'ste', 'hodili', 'van'] ['da', 'neki', 'ustali', 'kakovu', 'i', 'zvizde', 'i']
['kad', 'si', 'to', 'kad', 'si', 'spominjala', 'da', 'vas', 'je', 'u', 'selu', 'bilo', 'čuda', 'ča', 'so', 'većinon', 'ti', 'judi', 'ustali', 'tamo', 'kako', 'so', 'ste', 'so', 'pole', 'uni', 'šli', 'daje', 'n

['do', 'sad', 'kat', 'si', 'ti', 'za', 'katedrom', 'bimo', 'rekli', 'koliko', 'se', 'gambijalo'] ['drugu', 'san', 'kad', 'se', 'si', 'za']
['da', 't', 'to', 'si', 'liepo', 'rekla', 'je', 'krijanca', 'to', 'naj', 'najveć', 'to', 'je', 'ot', 'doma'] ['ili', 'znan', 'finili', 'put', 'nisan', 'ja', 'to', 'dica', 'znale', 'ma', 'baren', 'mi', 'se', 'para', 'da', 'smo', 'bili', 'imamo', 'poslu', 'šli', 'imalo', 'pesto', 'je', 'i', 'prema', 'delu', 'učitelja', 'da', 'si', 'ni', 'jih', 'nijanse', 'i', 'valje', 'njiga', 'da', 'si', 'poslušal', 'ono', 'če', 'ti', 'reka', 'pa', 'sada', 'mene', 'se', 'para', 'da', 'i', 'ja', 'više', 'nis', 'strahon', 'ne', 'nonine', 'k', 'e', 'dan', 'planetu', 'sin', 'va', 'načinila', 'ako', 'ma']
['da', 'da', 'da', 'da', 'dobro'] ['ni', 'ja', 'dobro']
['tako', 'u', 'slobodno', 'vrieme'] ['dan', 'kakovu', 'sobu', 'govorin', 'ne']
['š', 'čin', 'si', 'se', 'bavila', 'prveje', 'i', 'čin', 'sada', 'kad', 'najdeš', 'slobodnega', 'hipa'] ['šti', 'su', 'se', 'va', 'bilo'

['negdere', 'va', 'boško', 's', 'mojen', 'brekuon'] ['']
['i', 'ja', 'to', 'mi', 'je', 'vero', 'gušt'] ['']
['ma', 'ča', 'ja', 'znan', 'nekega', 'mješanca'] ['']
['neki', 'stafford', 'liči', 'onako', 'svi', 'ga', 'jako', 'imajo', 'straha'] ['']
['a', 'uon', 'ti', 'ne', 'bi', 'nanke', 'ne', 'znan'] ['']
['a', 'svi', 'ga', 'na', 'prvi', 'pogled', 'uh', 'tajsti', 'će', 'nas', 'pojes!'] ['']
['ma', 'vajk', 'smo', 'hodili', 'na', 'bartuljo', 'ja', 'znači', 'sveti', 'bartol', 'je', 'na', 'dvajset', 'i', 'četiri', 'uosmega', 'i', 'fešta', 'pokle', 'se', 'kako', 'drži', 'zadnji', 'vikend', 'va', 'uosmen', 'mesece'] ['']
['dok', 'smo', 'bili', 'minji', 'se', 'domišljan', 'to', 'mi', 'je', 'bilo', 'nešto', 'posebno', 'da', 'so', 'tamo', 'zad', 'kaštela', 'pekli', 'nekega', 'vola'] ['']
['i', 'to', 'bi', 'se', 'bilo', 'kroz', 'cieli', 'dan', 'pojelo', 'i', 'razdielilo', 'ljudima'] ['to', 'bi', 'se', 'bilo', 'prosti', 'jeli', 'dana', 'od', 'pojelo', 'je', 'sadil', 'jelo', 'ljudima', 'ah']
['a', 's

[(0.0, 0, 0, 0, 1, 1),
 (157.14285714285714, 1, 6, 4, 0, 7),
 (130.76923076923077, 3, 9, 5, 1, 13),
 (80.76923076923077, 1, 17, 3, 8, 26),
 (200.0, 0, 1, 1, 0, 1),
 (114.28571428571428, 5, 8, 3, 1, 14),
 (676.6666666666666, 5, 17, 181, 8, 30),
 (3400.0, 0, 1, 33, 0, 1),
 (235.29411764705884, 0, 30, 50, 4, 34),
 (117.64705882352942, 0, 16, 4, 1, 17),
 (389.6551724137931, 6, 15, 92, 8, 29),
 (269.5652173913044, 1, 19, 42, 3, 23),
 (17100.0, 0, 0, 171, 1, 1),
 (146.66666666666666, 1, 13, 8, 1, 15),
 (144.44444444444443, 3, 6, 4, 0, 9),
 (1300.0, 0, 1, 12, 0, 1),
 (8200.0, 0, 1, 81, 0, 1),
 (1800.0, 1, 1, 34, 0, 2),
 (200.0, 0, 1, 1, 0, 1),
 (110.00000000000001, 2, 4, 5, 4, 10),
 (140.0, 0, 9, 5, 1, 10),
 (125.0, 0, 3, 2, 1, 4),
 (7700.0, 0, 1, 76, 0, 1),
 (1450.0, 2, 1, 55, 1, 4),
 (125.0, 3, 13, 4, 0, 16),
 (100.0, 2, 7, 2, 2, 11),
 (493.33333333333337, 1, 12, 61, 2, 15),
 (120.0, 0, 4, 2, 1, 5),
 (225.0, 0, 4, 5, 0, 4),
 (169.23076923076923, 0, 12, 10, 1, 13),
 (3200.0, 0, 1, 31, 0, 1),

In [162]:
df_bench_1['wer'] = get_df_align_and_wer_info(df_bench_1, 'ckm009-2023-05-26-Žminj_02')
df_bench_1

,index,text_clean,xmin,xmax,transcription,wer
0,0,[dobro],0.061304,0.586118,[dobro],"(0.0, 0, 0, 0, 1, 1)"
1,1,"[kad, ti, spomenula, si, i, prietele, tako]",0.923131,4.116103,"[san, van, zi, rike, nekako]","(157.14285714285714, 1, 6, 4, 0, 7)"
2,2,"[kako, je, bilo, ovaj, duoći, tako, iz, jenega...",4.352784,7.829583,"[kat, bi, bilo, va, vrtu, čeka, mesto, već, ig...","(130.76923076923077, 3, 9, 5, 1, 13)"
3,3,"[si, rekla, da, su, ti, prieteli, jako, falili...",7.829583,15.966789,"[rekla, da, su, jako, farbi, preveć, su, zijal...","(80.76923076923077, 1, 17, 3, 8, 26)"
4,4,"[š, njima]",16.276344,17.197341,[eh],"(200.0, 0, 1, 1, 0, 1)"
...,...,...,...,...,...,...
114,114,"[i, na, kraje, se, to, pretvorilo, va, samo, g...",554.664054,558.723308,"[ima, trajset, to, pretvori, ala, va, samo, vi...","(100.0, 2, 4, 4, 4, 10)"
115,115,"[ali, va, štracarije]",558.723308,560.535199,"[ali, dva, prasca, je]","(133.33333333333331, 1, 1, 2, 1, 3)"
116,116,"[te, druge, fešti, baren, imajo, nešto, tradic...",561.379230,575.416580,"[mat, je, druge, friški, baren, ima, ja, nešto...","(106.89655172413792, 5, 8, 18, 16, 29)"
117,117,"[a, tamo, samo, te, štraci, ti, štrafanići]",575.797327,579.085003,"[a, tamo, samo, te, špijat, sjetiš, ten, san, ...","(114.28571428571428, 0, 3, 5, 4, 7)"


In [163]:
df_bench_1['wer_value'] = df_bench_1['wer'].apply(lambda x: x[0])
df_bench_1

,index,text_clean,xmin,xmax,transcription,wer,wer_value
0,0,[dobro],0.061304,0.586118,[dobro],"(0.0, 0, 0, 0, 1, 1)",0.000000
1,1,"[kad, ti, spomenula, si, i, prietele, tako]",0.923131,4.116103,"[san, van, zi, rike, nekako]","(157.14285714285714, 1, 6, 4, 0, 7)",157.142857
2,2,"[kako, je, bilo, ovaj, duoći, tako, iz, jenega...",4.352784,7.829583,"[kat, bi, bilo, va, vrtu, čeka, mesto, već, ig...","(130.76923076923077, 3, 9, 5, 1, 13)",130.769231
3,3,"[si, rekla, da, su, ti, prieteli, jako, falili...",7.829583,15.966789,"[rekla, da, su, jako, farbi, preveć, su, zijal...","(80.76923076923077, 1, 17, 3, 8, 26)",80.769231
4,4,"[š, njima]",16.276344,17.197341,[eh],"(200.0, 0, 1, 1, 0, 1)",200.000000
...,...,...,...,...,...,...,...
114,114,"[i, na, kraje, se, to, pretvorilo, va, samo, g...",554.664054,558.723308,"[ima, trajset, to, pretvori, ala, va, samo, vi...","(100.0, 2, 4, 4, 4, 10)",100.000000
115,115,"[ali, va, štracarije]",558.723308,560.535199,"[ali, dva, prasca, je]","(133.33333333333331, 1, 1, 2, 1, 3)",133.333333
116,116,"[te, druge, fešti, baren, imajo, nešto, tradic...",561.379230,575.416580,"[mat, je, druge, friški, baren, ima, ja, nešto...","(106.89655172413792, 5, 8, 18, 16, 29)",106.896552
117,117,"[a, tamo, samo, te, štraci, ti, štrafanići]",575.797327,579.085003,"[a, tamo, samo, te, špijat, sjetiš, ten, san, ...","(114.28571428571428, 0, 3, 5, 4, 7)",114.285714


In [159]:
file_name = 'ckm009-2023-05-26-Žminj_02'
df_wer = pd.read_csv('/users/irisz/downloads/elpis_result/' + file_name + '_wer.csv')
df_wer

,Unnamed: 0,manual,model,model_fuzzy,error_type
0,0,dobro,dobro,dobro,c
1,1,kad,-,-,d
2,2,ti,-,-,d
3,3,spomenula,-,-,d
4,4,si,-,-,d
...,...,...,...,...,...
2619,2619,mi,-,i,s
2620,2620,baš,-,vaš,s
2621,2621,jako,jako,jako,c
2622,2622,to,to,to,c


In [160]:
df_wer.head(40)

,Unnamed: 0,manual,model,model_fuzzy,error_type
0,0,dobro,dobro,dobro,c
1,1,kad,-,-,d
2,2,ti,-,-,d
3,3,spomenula,-,-,d
4,4,si,-,-,d
5,5,i,-,zi,s
6,6,prietele,-,-,d
7,7,tako,-,-,d
8,8,-,san,-,i
9,9,-,van,-,i
